<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Algorithmic Trading 

**Chapter 05 &mdash; Predicting Market Movements with Machine Learning**

## Using Linear Regression for Market Movement Prediction

### A Quick Review of Linear Regression

In [ ]:
import numpy as np

In [ ]:
%matplotlib inline
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'

In [ ]:
x = np.linspace(0, 10)

In [ ]:
np.random.seed(100)

In [ ]:
y = x + np.random.standard_normal(len(x))

In [ ]:
reg = np.polyfit(x, y, deg=1)

In [ ]:
reg

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, y, 'bo', label='data')
plt.plot(x, np.polyval(reg, x), 'r', lw=2.5,
         label='linear regression')
plt.legend(loc=0);
# plt.savefig('../../images/ch05/lr_plot_1.png')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, y, 'bo', label='data')
xn = np.linspace(0, 20)
plt.plot(xn, np.polyval(reg, xn), 'r', lw=2.5,
         label='linear regression')
plt.legend(loc=0);
# plt.savefig('../../images/ch05/lr_plot_2.png')

### The Basic Idea for Price Prediction

In [ ]:
x = np.arange(12)

In [ ]:
x

In [ ]:
lags = 3

In [ ]:
m = np.zeros((lags + 1, len(x) - lags))

In [ ]:
m[lags] = x[lags:]
for i in range(lags):
    m[i] = x[i:i - lags]

In [ ]:
m.T

In [ ]:
reg = np.linalg.lstsq(m[:lags].T, m[lags], rcond=None)[0]

In [ ]:
reg

In [ ]:
np.dot(m[:lags].T, reg)

### Predicting Index Levels

In [ ]:
import pandas as pd

In [ ]:
raw = pd.read_csv('http://hilpisch.com/tr_eikon_eod_data.csv',
                  index_col=0, parse_dates=True).dropna()

In [ ]:
data = pd.DataFrame(raw['.SPX'])

In [ ]:
data.rename(columns={'.SPX': 'price'}, inplace=True)

In [ ]:
lags = 3

In [ ]:
cols = []
for lag in range(1, lags + 1):
    col = f'lag_{lag}'
    data[col] = data['price'].shift(lag) # <1>
    cols.append(col)
data.dropna(inplace=True)

In [ ]:
reg = np.linalg.lstsq(data[cols], data['price'],
                      rcond=None)[0]

In [ ]:
reg

In [ ]:
data['prediction'] = np.dot(data[cols], reg)

In [ ]:
data[['price', 'prediction']].plot(figsize=(10, 6));
# plt.savefig('../../images/ch05/lr_plot_3.png')

In [ ]:
data[['price', 'prediction']].loc['2018-3-1':].plot(
            figsize=(10, 6));
# plt.savefig('../../images/ch05/lr_plot_4.png')

### Predicting Future Returns

In [ ]:
data['returns'] = np.log(data['price'] /
                         data['price'].shift(1))

In [ ]:
data.dropna(inplace=True)

In [ ]:
cols = []
for lag in range(1, lags + 1):
    col = f'lag_{lag}'
    data[col] = data['returns'].shift(lag) # <3>
    cols.append(col)
data.dropna(inplace=True)

In [ ]:
reg = np.linalg.lstsq(data[cols], data['returns'],
                      rcond=None)[0]

In [ ]:
reg

In [ ]:
data['prediction'] = np.dot(data[cols], reg)

In [ ]:
data[['returns', 'prediction']].iloc[lags:].plot(figsize=(10, 6));
# plt.savefig('../../images/ch05/lr_plot_5.png')

In [ ]:
hits = np.sign(data['returns'] *
               data['prediction']).value_counts()

In [ ]:
hits

In [ ]:
hits.values[0] / sum(hits)

### Prediction Future Market Direction

In [ ]:
reg = np.linalg.lstsq(data[cols], np.sign(data['returns']),
                      rcond=None)[0]

In [ ]:
reg

In [ ]:
data['prediction'] = np.sign(np.dot(data[cols], reg))

In [ ]:
data['prediction'].value_counts()

In [ ]:
hits = np.sign(data['returns'] *
               data['prediction']).value_counts()

In [ ]:
hits

In [ ]:
hits.values[0] / sum(hits)

### Vectorized Backtesting of Regression-based Strategy 

In [ ]:
data.head()

In [ ]:
data['strategy'] = data['prediction'] * data['returns']

In [ ]:
data[['returns', 'strategy']].dropna().cumsum(
        ).apply(np.exp).plot(figsize=(10, 6));
# plt.savefig('../../images/ch05/lr_plot_6.png')

### Generalizing the Approach

In [ ]:
import LRVectorBacktester as LR

In [ ]:
lrbt = LR.LRVectorBacktester('.SPX', '2010-1-1', '2018-06-29',
                                     10000, 0.0)

In [ ]:
lrbt.run_strategy('2010-1-1', '2018-01-01',
                  '2010-1-1', '2018-01-01', lags=3)

In [ ]:
lrbt.run_strategy('2010-1-1', '2015-12-31',
                  '2015-1-1', '2018-01-01', lags=3)

In [ ]:
lrbt.plot_results()
# plt.savefig('../../images/ch05/lr_plot_7.png')

In [ ]:
lrbt = LR.LRVectorBacktester('GDX', '2010-1-1', '2018-06-29',
                                     10000, 0.001)

In [ ]:
lrbt.run_strategy('2010-1-1', '2015-12-31',
                  '2016-1-1', '2018-06-29', lags=5)

In [ ]:
lrbt.plot_results()
# plt.savefig('../../images/ch05/lr_plot_8.png')

## Using Machine Learning for Market Movement Prediction

### Linear Regression with scikit-learn

In [ ]:
x = np.arange(12)

In [ ]:
x

In [ ]:
lags = 3

In [ ]:
m = np.zeros((lags + 1, len(x) - lags))

In [ ]:
m[lags] = x[lags:]
for i in range(lags):
    m[i] = x[i:i - lags]

In [ ]:
from sklearn import linear_model

In [ ]:
lm = linear_model.LinearRegression()

In [ ]:
lm.fit(m[:lags].T, m[lags])

In [ ]:
lm.coef_

In [ ]:
lm.intercept_

In [ ]:
lm.predict(m[:lags].T)

In [ ]:
lm = linear_model.LinearRegression(fit_intercept=False)

In [ ]:
lm.fit(m[:lags].T, m[lags])

In [ ]:
lm.coef_

In [ ]:
lm.intercept_

In [ ]:
lm.predict(m[:lags].T)

### A Simple Classification Problem

In [ ]:
hours = np.array([0.5, 0.75, 1., 1.25, 1.5, 1.75, 1.75, 2.,
                  2.25, 2.5, 2.75, 3., 3.25, 3.5, 4., 4.25,
                  4.5, 4.75, 5., 5.5])

In [ ]:
success = np.array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
                    0, 1, 1, 1, 1, 1, 1])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(hours, success, 'ro')
plt.ylim(-0.2, 1.2);
# plt.savefig('../../images/ch05/ml_plot_1.png')

In [ ]:
reg = np.polyfit(hours, success, deg=1)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(hours, success, 'ro')
plt.plot(hours, np.polyval(reg, hours), 'b')
plt.ylim(-0.2, 1.2);
# plt.savefig('../../images/ch05/ml_plot_2.png')

In [ ]:
lm = linear_model.LogisticRegression(solver='lbfgs')

In [ ]:
hrs = hours.reshape(1, -1).T

In [ ]:
lm.fit(hrs, success)

In [ ]:
prediction = lm.predict(hrs)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(hours, success, 'ro', label='data')
plt.plot(hours, prediction, 'b', label='prediction')
plt.legend(loc=0)
plt.ylim(-0.2, 1.2);
# plt.savefig('../../images/ch05/ml_plot_3.png')

In [ ]:
prob = lm.predict_proba(hrs)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(hours, success, 'ro')
plt.plot(hours, prediction, 'b')
plt.plot(hours, prob.T[0], 'm--',
         label='$p(h)$ for zero')
plt.plot(hours, prob.T[1], 'g-.',
         label='$p(h)$ for one')
plt.ylim(-0.2, 1.2)
plt.legend(loc=0)
# plt.savefig('../../images/ch05/ml_plot_4.png')

### Using Logistic Regression to Predict Market Direction

In [ ]:
data = pd.read_csv('http://hilpisch.com/tr_eikon_eod_data.csv',
                  index_col=0, parse_dates=True).dropna()

In [ ]:
data = pd.DataFrame(data['.SPX'])

In [ ]:
data.rename(columns={'.SPX': 'price'}, inplace=True)

In [ ]:
data['returns'] = np.log(data['price'] / data['price'].shift(1))

In [ ]:
data.dropna(inplace=True)

In [ ]:
lags = 3

In [ ]:
cols = []
for lag in range(1, lags + 1):
    col = 'lag_{}'.format(lag)
    data[col] = data['returns'].shift(lag)
    cols.append(col)

In [ ]:
data.dropna(inplace=True)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
lm = linear_model.LogisticRegression(C=1e6, solver='lbfgs',
                                     multi_class='auto')

In [ ]:
lm.fit(data[cols], np.sign(data['returns']))

In [ ]:
data['prediction'] = lm.predict(data[cols])

In [ ]:
data['prediction'].value_counts()

In [ ]:
hits = np.sign(data['returns'].iloc[lags:] *
               data['prediction'].iloc[lags:]
              ).value_counts()

In [ ]:
hits

In [ ]:
accuracy_score(data['prediction'],
               np.sign(data['returns']))

In [ ]:
data['strategy'] = data['prediction'] * data['returns']

In [ ]:
data[['returns', 'strategy']].cumsum().apply(np.exp).plot(
                                        figsize=(10, 6));
# plt.savefig('../../images/ch05/ml_plot_5.png')

In [ ]:
lags = 15

In [ ]:
cols = []
for lag in range(1, lags + 1):
    col = 'lag_%d' % lag
    data[col] = data['returns'].shift(lag)
    cols.append(col)

In [ ]:
data.dropna(inplace=True)

In [ ]:
lm.fit(data[cols], np.sign(data['returns']))

In [ ]:
data['prediction'] = lm.predict(data[cols])

In [ ]:
data['prediction'].value_counts()

In [ ]:
hits = np.sign(data['returns'].iloc[lags:] *
               data['prediction'].iloc[lags:]
              ).value_counts()

In [ ]:
hits

In [ ]:
accuracy_score(data['prediction'],
               np.sign(data['returns']))

In [ ]:
data['strategy'] = data['prediction'] * data['returns']

In [ ]:
data[['returns', 'strategy']].cumsum().apply(np.exp).plot(
                                        figsize=(10, 6));
# plt.savefig('../../images/ch05/ml_plot_6.png')

### Generalizing the Approach

In [ ]:
import ScikitVectorBacktester as SCI

In [ ]:
scibt = SCI.ScikitVectorBacktester('.SPX',
                                   '2010-1-1', '2018-06-29',
                                   10000, 0.0, 'logistic')

In [ ]:
scibt.run_strategy('2010-1-1', '2018-06-29',
                   '2010-1-1', '2018-06-29', lags=15)

In [ ]:
scibt.run_strategy('2010-1-1', '2013-12-31',
                   '2014-1-1', '2018-06-29', lags=15)

In [ ]:
scibt.plot_results()
# plt.savefig('../../images/ch05/ml_plot_7.png')

In [ ]:
scibt = SCI.ScikitVectorBacktester('.SPX',
                                   '2010-1-1', '2018-06-29',
                                   10000, 0.001, 'logistic')

In [ ]:
scibt.run_strategy('2010-1-1', '2013-12-31',
                   '2014-1-1', '2016-10-31', lags=15)

In [ ]:
scibt.plot_results()
# plt.savefig('../../images/ch05/ml_plot_8.png')

## Using Deep Learning for Market Movement Prediction

#### The Simple Classification Problem Revisited 

In [ ]:
hours = np.array([0.5, 0.75, 1., 1.25, 1.5, 1.75, 1.75, 2.,
                  2.25, 2.5, 2.75, 3., 3.25, 3.5, 4., 4.25,
                  4.5, 4.75, 5., 5.5])

In [ ]:
success = np.array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
                    0, 1, 1, 1, 1, 1, 1]) 

In [ ]:
data = pd.DataFrame({'hours': hours, 'success': success})

In [ ]:
data.info()

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
model = MLPClassifier(hidden_layer_sizes=[32],
                     max_iter=1000)

In [ ]:
model.fit(data['hours'].values.reshape(-1, 1), data['success'])

In [ ]:
data['prediction'] = model.predict(data['hours'].values.reshape(-1, 1)) # <5>

In [ ]:
data.tail()

In [ ]:
data.plot(x='hours', y=['success', 'prediction'],
          style=['ro', 'b-'], ylim=[-.1, 1.1],
          figsize=(10, 6));
# plt.savefig('../../images/ch05/dl_plot_1.png')

### Using Deep Neural Networks to Predict Market Direction

In [ ]:
data = pd.read_csv('http://hilpisch.com/tr_eikon_eod_data.csv',
                  index_col=0, parse_dates=True).dropna()

In [ ]:
data = pd.DataFrame(data['.SPX'])

In [ ]:
data.rename(columns={'.SPX': 'price'}, inplace=True)

In [ ]:
data['returns'] = np.log(data['price'] /
                         data['price'].shift(1)) 

In [ ]:
data['direction'] = np.where(data['returns'] > 0, 1, 0)

In [ ]:
lags = 5

In [ ]:
cols = []
for lag in range(1, lags + 1): # <5>
    col = 'ret_{}'.format(lag)
    data[col] = data['direction'].shift(lag) # <6>
    cols.append(col)
data.dropna(inplace=True) # <7>
data[cols] = data[cols].astype(int)

In [ ]:
print(data.round(4).tail())

In [ ]:
np.random.seed(500)
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
tf.random.set_random_seed(500)
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu',
                input_shape=(lags,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # <5>
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
cutoff = '2015-1-1'

In [ ]:
training_data = data[data.index < cutoff].copy()

In [ ]:
test_data = data[data.index >= cutoff].copy()

In [ ]:
%%time
model.fit(training_data[cols],
          training_data['direction'],
          epochs=100, verbose=False)

In [ ]:
pred = model.predict_classes(training_data[cols])

In [ ]:
pred[:30].flatten()

In [ ]:
training_data['prediction'] = np.where(pred > 0, 1, -1)

In [ ]:
training_data['strategy'] = (training_data['prediction'] *
                            training_data['returns'])

In [ ]:
training_data[['returns', 'strategy']].cumsum(
                ).apply(np.exp).plot(figsize=(10, 6));
# plt.savefig('../../images/ch05/dl_plot_2.png')

In [ ]:
pred = model.predict_classes(test_data[cols])

In [ ]:
test_data['prediction'] = np.where(pred > 0, 1, -1)

In [ ]:
test_data['strategy'] = (test_data['prediction'] *
                        test_data['returns'])

In [ ]:
test_data[['returns', 'strategy']].cumsum(
                ).apply(np.exp).plot(figsize=(10, 6));
# plt.savefig('../../images/ch05/dl_plot_3.png')

### Adding Different Types of Features

In [ ]:
def transform(x):
    if x < -0.0015: return -2
    elif x < 0: return -1
    elif x > 0.0015: return 2
    else: return 1

In [ ]:
data['momentum'] = data['returns'].rolling(4).mean().shift(1)

In [ ]:
data['momentum'] = data['momentum'].apply(transform)

In [ ]:
data.dropna(inplace=True)

In [ ]:
cols.append('momentum')

In [ ]:
print(data.round(4).tail())

In [ ]:
training_data = data[data.index < cutoff].copy()

In [ ]:
test_data = data[data.index >= cutoff].copy()

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu',
                input_shape=(len(cols),)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
%%time 
model.fit(training_data[cols], training_data['direction'],
          verbose=False, epochs=100)

In [ ]:
pred = model.predict_classes(training_data[cols])

In [ ]:
training_data['prediction'] = np.where(pred > 0, 1, -1)

In [ ]:
training_data['strategy'] = training_data['prediction'] * \
                            training_data['returns']

In [ ]:
training_data[['returns', 'strategy']].cumsum(
                ).apply(np.exp).plot(figsize=(10, 6));
# plt.savefig('../../images/ch05/dl_plot_4.png')

In [ ]:
pred = model.predict_classes(test_data[cols])

In [ ]:
test_data['prediction'] = np.where(pred > 0, 1, -1)

In [ ]:
test_data['strategy'] = (test_data['prediction'] *
                        test_data['returns'])

In [ ]:
test_data[['returns', 'strategy']].cumsum(
                ).apply(np.exp).plot(figsize=(10, 6));
# plt.savefig('../../images/ch05/dl_plot_5.png')

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>